Read Mesh

In [1]:
from lnas import LnasFormat
import pathlib

meshes_path = pathlib.Path("../insight/Docker/local/volume/PrologisCajamar4_full/G100/mesh")

mesh = LnasFormat.from_file(meshes_path / "G100.lnas")
negative_mesh = LnasFormat.from_file(meshes_path / "G100_negative.lnas")

Read historic series

In [2]:
import pandas as pd
import pathlib

data_path = pathlib.Path("../insight/Docker/local/volume/PrologisCajamar4_full/G100/cases/013/data/body_data")

body_hist_series = pd.read_hdf(data_path / "bodies.G100_hs.data.h5")
negative_body_hist_series = pd.read_hdf(data_path / "bodies.G100_neg_hs.data.h5")

last_index = body_hist_series.point_idx.max()
negative_body_hist_series.point_idx += last_index + 1

/tmp/ipykernel_13995/3235872955.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


: 

Offset body

In [ ]:
import numpy as np

NORMAL_OFFSET = - 0.125


for surface_name in negative_mesh.surfaces.keys():
    tri_normals = negative_mesh.geometry_from_surface(surface_name).normals.copy()
    tri_verts = negative_mesh.geometry_from_surface(surface_name).triangle_vertices.copy()
    tri_verts += np.repeat(tri_normals, 3, axis=0).reshape(-1, 3, 3) * NORMAL_OFFSET

    new_verts = tri_verts.copy().reshape(-1, 3)
    new_verts = np.unique(new_verts, axis=0)

    new_triangles = []

    for tri in tri_verts:
        tri_index = []
        for v in tri:
            matches = np.all(new_verts == v, axis=1)
            indices = np.where(matches)
            tri_index.append(indices[0][0])
        tri_index.reverse()
        new_triangles.append(np.array(tri_index))

    new_triangles = np.array(new_triangles, dtype=np.uint32) + len(mesh.geometry.vertices)

    new_tri_indices = np.arange(
        mesh.geometry.triangles.shape[0], mesh.geometry.triangles.shape[0] + new_triangles.shape[0]
    )
    mesh.surfaces[surface_name + "_inside"] = new_tri_indices

    mesh.geometry.vertices = np.concatenate((mesh.geometry.vertices, new_verts))
    mesh.geometry.triangles = np.concatenate((mesh.geometry.triangles, new_triangles))

    mesh.geometry._update_triangles_vertices()
    mesh.geometry._update_normals()
    mesh.geometry._update_areas()
    mesh.geometry._update_vertices_normals()


filename = meshes_path / "G100.merged.lnas"
export_path = meshes_path / "G100.merged.stl"

mesh.to_file(filename)
mesh.export_stl(export_path)

Join historic series

In [ ]:
new_df = pd.concat([body_hist_series, negative_body_hist_series]).sort_values(
    by=["time_step", "point_idx"]
)
new_df.to_hdf(path_or_buf=data_path / "bodies.G100.merged.data.h5", key="df", mode="w")

new_df

: 